# TMDD Benchmarking
## Data generation

This notebook benchmarks the PySaem implementation on reference data.

The model is a target-mediated drug disposition (TMDD) model, described for instance in [Dua et al. (2015)](https://doi.org/10.1002/psp4.41), that contains 3 ODEs:

```math
\begin{align*}
\frac{dL}{dt} =& -k_{eL} L -k_{on} LR + k_{off} P, \\
\frac{dR}{dt} =& k_{syn} - k_{def} R -k_{on} LR +k_{off} P, \\
\frac{dP}{dt} =& k_{on} LR + k_{off} P - k_{eP} P, \\
&L(0) = L_0, R(0) = R_0 = \frac{k_{syn}}{k_{deg}}, P(0) = 0, \\
&\frac{k_{off}}{k_{on}} = K_D
\end{align*}
```

## Reference parameter values
The parameters' reference values and distributions are gathered in the table below. Distributed parameters follow a log-normal distribution.

| Parameter | Description | Assumption | Reference value | Spread (log) |
| --- | --- | --- | ---| ---|
| $`k_{off}`$ | Unbinding constant rate | Known | $`1 h^{-1}`$ | None |
| $`k_{D}`$  | Dissociation constant | Known | $`1 nM`$ |  None |
| $`k_{on}`$  | On-rate constant | Known | $`\frac{k_{off}}{k_D}`$ |  None |
| $`inj`$ | Administered concentration | Known | $`10 nanomole`$ | None |
| $`V_c`$ | Distribution volume | Unknown | $`3L`$ | $`\omega^2 = 0.1`$ |
| $`L_0`$ | Initial antibody concentration | Known | $`10 nM`$ | None |
| $`k_{deg}`$  | Receptor elimination rate | Known | $`1e-2 h^{-1}`$ | None |
| $`R_0`$  | Total receptor concentration | Unknown | $`3 nM`$ | $` \omega^2 = 0.1 `$ |
| $`k_{syn}`$  | Receptor synthesis rate | Known | $`R_0 \times k_{deg}`$ |  None |
| $`k_{eL}`$  | Free antibody elimination rate | Unknown | $`5e-2 h^{-1}`$ | $`\omega^2 = 0.5`$ |
| $`k_{eP}`$  | Bound antibody elimination rate | Unknown | $`1e-1 h^{-1}`$ | $` \omega^2 = 0.1 `$ |

In [ ]:
import numpy as np
import pandas as pd
import uuid
import pickle
from IPython.display import display


%load_ext autoreload
%autoreload 2

from vpop_calibration import *
from examples.benchmarking.tmdd.generate_tmdd_data import generate_benchmark_data

In [ ]:
obs_df, patients_df, tmdd_model, time_steps, protocol_design, output_names = (
    generate_benchmark_data()
)

# Create a training data set

In [ ]:
log_nb_patients = 10
param_ranges = {
    "k_eL": {"low": -4.0, "high": 0.0, "log": True},
    "k_eP": {"low": -4.0, "high": 0.0, "log": True},
    "R0": {"low": -1, "high": 3.0, "log": True},
    "Vc": {"low": 0.0, "high": 4.0, "log": True},
}

training_data = simulate_dataset_from_ranges(
    tmdd_model, log_nb_patients, param_ranges, time_steps, protocol_design
)
training_data = training_data.loc[training_data["output_name"].isin(output_names)]
display(training_data)
ode_params = list(param_ranges.keys())
descriptors = ode_params + ["time"]

# Train the GP

In [ ]:
myGP = GP(
    training_data,
    descriptors,
    var_strat="IMV",  # either IMV (Independent Multitask Variational) or LMCV (Linear Model of Coregionalization Variational)
    kernel="RBF",  # Either RBF or SMK
    deep_kernel=True,
    nb_inducing_points=50,
    nb_training_iter=200,
    training_proportion=0.7,
    learning_rate=0.1,
    lr_decay=0.99,
    nb_features=10,
    nb_latents=3,
    log_inputs=ode_params,
    log_outputs=output_names,
    min_delta=1e-3,
)
# Train the GP
myGP.train()

In [ ]:
myGP.plot_obs_vs_predicted("training", (5, 5))

In [ ]:
# NLME model parameters
init_log_MI = {}
init_log_PDU = {
    "k_eL": {"mean": -1.0, "sd": 0.5},
    "k_eP": {"mean": -1.0, "sd": 0.5},
    "R0": {"mean": 0.0, "sd": 0.5},
    "Vc": {"mean": 0.0, "sd": 0.5},
}

error_model_type = "proportional"
init_res_var = [0.1]

# Create a structural model
structural_gp = StructuralGp(myGP)
# Create a NLME model
nlme_surrogate = NlmeModel(
    structural_gp,
    patients_df,
    init_log_MI,
    init_log_PDU,
    init_res_var,
    None,
    error_model_type,
    pred_var_threshold=1e-2,
    num_chains=1,
    constraints=structural_gp.training_ranges,
)
optimizer = PySaem(
    nlme_surrogate,
    obs_df,
    nb_phase1_iterations=500,
    nb_phase2_iterations=200,
    mcmc_nb_transitions=1,
    mcmc_first_burn_in=20,
    verbose=False,
    plot_frames=50,
)
optimizer.run()

In [ ]:
_ = check_surrogate_validity_gp(nlme_surrogate)

In [ ]:
plot_map_estimates(nlme_surrogate)